In [1]:

import re
import numpy as np
import pandas as pd
from Bio import Seq
from Bio.Seq import translate
from Bio.Data.CodonTable import standard_dna_table


In [2]:
rev_table = {}
for codon, aa in standard_dna_table.forward_table.items():
    rev_table.setdefault(aa, []).append(codon)

rev_table


{'F': ['TTT', 'TTC'],
 'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
 'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
 'Y': ['TAT', 'TAC'],
 'C': ['TGT', 'TGC'],
 'W': ['TGG'],
 'P': ['CCT', 'CCC', 'CCA', 'CCG'],
 'H': ['CAT', 'CAC'],
 'Q': ['CAA', 'CAG'],
 'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
 'I': ['ATT', 'ATC', 'ATA'],
 'M': ['ATG'],
 'T': ['ACT', 'ACC', 'ACA', 'ACG'],
 'N': ['AAT', 'AAC'],
 'K': ['AAA', 'AAG'],
 'V': ['GTT', 'GTC', 'GTA', 'GTG'],
 'A': ['GCT', 'GCC', 'GCA', 'GCG'],
 'D': ['GAT', 'GAC'],
 'E': ['GAA', 'GAG'],
 'G': ['GGT', 'GGC', 'GGA', 'GGG']}

In [3]:
clinvar_df = pd.read_csv("clinvar_result.txt", sep="\t")
clinvar_df


/tmp/ipykernel_27525/3134801216.py:1: DtypeWarning: Columns (18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  clinvar_df = pd.read_csv("clinvar_result.txt", sep="\t")


,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
0,GRCh37/hg19 1q21.1(chr1:143134063-143284670)x3,NaN,NaN,See cases,VCV000145208,1,143134063 - 143284670,1,16302 - 166909,145208,...,Benign,"Nov 30, 2010",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),OR4F5,E36G,not specified,VCV002205837,1,69134,1,69134,2205837,...,Likely benign,"Aug 23, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRCh37/hg19 1p36.11(chr1:26930197-26968251)x1,LOC129929833|LOC129929834|LOC129929835,NaN,See cases,VCV000151336,1,26930197 - 26968251,1,91324 - 128277,151336,...,Likely benign,"Apr 30, 2011",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GRCh38/hg38 1p36.33-35.1(chr1:99125-34026935)x3,A2ML1-AS2|A3GALT2|AADACL3|AADACL4|ACAP3|ACOT7|...,NaN,Trisomy 12p,VCV002664871,NaN,NaN,1,99125 - 34026935,2664871,...,Pathogenic,"Dec 12, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GRCh38/hg38 1p36.33-36.23(chr1:629025-8537745)x1,LOC121967050|LOC121967051|LOC121967052|LOC1219...,NaN,See cases,VCV000058242,1,564405 - 8597804,1,629025 - 8537745,58242,...,Pathogenic,"Aug 12, 2011","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451376,"NF1, 80-KB DEL",NF1,NaN,Café-au-lait macules with pulmonary stenosis,VCV000000342,NaN,NaN,NaN,NaN,342,...,Pathogenic,"Dec 1, 1992",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451377,NG_009018.1:g.246621_246622insN[320],NF1,NaN,"Neurofibromatosis, type 1",VCV000000332,NaN,NaN,NaN,NaN,332,...,Pathogenic,"Oct 31, 1991",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451378,NM_004628.4(XPC):c.621_622ins83 (p.?),XPC,NaN,"Xeroderma pigmentosum, group C",VCV000000254,NaN,NaN,NaN,NaN,254,...,Pathogenic,"Dec 1, 1993",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451379,NG_023416.2:g.(88604_176551)_(176714_576894)del,HPSE2,NaN,Urofacial syndrome type 1,VCV000000087,NaN,NaN,NaN,NaN,87,...,Pathogenic,"Jun 11, 2010",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
clinvar_df.head()


,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
0,GRCh37/hg19 1q21.1(chr1:143134063-143284670)x3,NaN,NaN,See cases,VCV000145208,1,143134063 - 143284670,1,16302 - 166909,145208,...,Benign,"Nov 30, 2010",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),OR4F5,E36G,not specified,VCV002205837,1,69134,1,69134,2205837,...,Likely benign,"Aug 23, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GRCh37/hg19 1p36.11(chr1:26930197-26968251)x1,LOC129929833|LOC129929834|LOC129929835,NaN,See cases,VCV000151336,1,26930197 - 26968251,1,91324 - 128277,151336,...,Likely benign,"Apr 30, 2011",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GRCh38/hg38 1p36.33-35.1(chr1:99125-34026935)x3,A2ML1-AS2|A3GALT2|AADACL3|AADACL4|ACAP3|ACOT7|...,NaN,Trisomy 12p,VCV002664871,NaN,NaN,1,99125 - 34026935,2664871,...,Pathogenic,"Dec 12, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GRCh38/hg38 1p36.33-36.23(chr1:629025-8537745)x1,LOC121967050|LOC121967051|LOC121967052|LOC1219...,NaN,See cases,VCV000058242,1,564405 - 8597804,1,629025 - 8537745,58242,...,Pathogenic,"Aug 12, 2011","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
clinvar_df.columns

Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Variant type', 'Molecular consequence',
       'Germline classification', 'Germline date last evaluated',
       'Germline review status', 'Somatic clinical impact',
       'Somatic clinical impact date last evaluated',
       'Somatic clinical impact review status', 'Oncogenicity classification',
       'Oncogenicity date last evaluated', 'Oncogenicity review status',
       'Unnamed: 24'],
      dtype='object')

In [6]:
clinvar_df.loc[1]

Name                                           NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly)
Gene(s)                                                                              OR4F5
Protein change                                                                        E36G
Condition(s)                                                                 not specified
Accession                                                                     VCV002205837
GRCh37Chromosome                                                                         1
GRCh37Location                                                                       69134
GRCh38Chromosome                                                                         1
GRCh38Location                                                                       69134
VariationID                                                                        2205837
AlleleID(s)                                                                        2193183

In [7]:
clinvar_df['Variant type'].unique()

array(['copy number gain', 'single nucleotide variant',
       'copy number loss', 'Deletion', 'Microsatellite', 'Indel',
       'Duplication', 'Inversion', 'Insertion', 'Haplotype', 'Variation',
       'Complex', 'fusion', 'Translocation', 'CompoundHeterozygote',
       'Phase unknown', 'Diplotype', 'protein only',
       'Distinct chromosomes'], dtype=object)

In [8]:
clinvar_df['Variant type'].value_counts()

Variant type
single nucleotide variant    1225608
Deletion                      115943
Duplication                    46917
Microsatellite                 22187
copy number loss               12354
copy number gain                9590
Insertion                       9572
Indel                           7303
Haplotype                        477
Inversion                        477
Variation                        436
CompoundHeterozygote             247
Translocation                    143
protein only                      74
Complex                           41
fusion                             5
Phase unknown                      5
Diplotype                          1
Distinct chromosomes               1
Name: count, dtype: int64

In [9]:
snv_clinvar_df = clinvar_df[clinvar_df['Variant type'] == 'single nucleotide variant']
snv_clinvar_df


,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
1,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),OR4F5,E36G,not specified,VCV002205837,1,69134,1,69134,2205837,...,Likely benign,"Aug 23, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NM_001385641.1(SAMD11):c.552C>T (p.Ile184=),SAMD11,NaN,not provided,VCV001543320,1,861336,1,925956,1543320,...,Likely benign,"Nov 20, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NM_001385641.1(SAMD11):c.565C>T (p.Pro189Ser),SAMD11,"P10S, P189S",not provided,VCV001648427,1,861349,1,925969,1648427,...,Likely benign,"Dec 9, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NM_001385641.1(SAMD11):c.576C>T (p.Cys192=),SAMD11,NaN,not provided,VCV001936488,1,861360,1,925980,1936488,...,Likely benign,"Mar 29, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NM_001385641.1(SAMD11):c.582C>T (p.Cys194=),SAMD11,NaN,not provided,VCV001568423,1,861366,1,925986,1568423,...,Likely benign,"Jul 5, 2022","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451320,"MTHFR, 1081C-T",MTHFR,NaN,Homocystinuria due to methylene tetrahydrofola...,VCV000003526,NaN,NaN,NaN,NaN,3526,...,Pathogenic,"Apr 24, 2000",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451331,"NPC1, IVS16, G-A, -82",NPC1,NaN,"Niemann-Pick disease, type C1",VCV000002975,NaN,NaN,NaN,NaN,2975,...,Pathogenic,"Jul 1, 2001",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451340,"NIPBL, IVS44DS, A-G, +4",NIPBL,NaN,Cornelia de Lange syndrome 1,VCV000002149,NaN,NaN,NaN,NaN,2149,...,Pathogenic,"Oct 1, 2004",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451349,A114N,NIPAL4,NaN,Autosomal recessive congenital ichthyosis 6,VCV000001730,NaN,NaN,NaN,NaN,1730,...,Pathogenic,"Oct 15, 2004",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
snv_clinvar_df.loc[:, 'VariationID'] = snv_clinvar_df['VariationID'].apply(lambda vid: f"ClinVar:{vid}")
snv_clinvar_df


/tmp/ipykernel_27525/3196568711.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['ClinVar:2205837' 'ClinVar:1543320' 'ClinVar:1648427' ... 'ClinVar:2149'
 'ClinVar:1730' 'ClinVar:351']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  snv_clinvar_df.loc[:, 'VariationID'] = snv_clinvar_df['VariationID'].apply(lambda vid: f"ClinVar:{vid}")


,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline classification,Germline date last evaluated,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24
1,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),OR4F5,E36G,not specified,VCV002205837,1,69134,1,69134,ClinVar:2205837,...,Likely benign,"Aug 23, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NM_001385641.1(SAMD11):c.552C>T (p.Ile184=),SAMD11,NaN,not provided,VCV001543320,1,861336,1,925956,ClinVar:1543320,...,Likely benign,"Nov 20, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NM_001385641.1(SAMD11):c.565C>T (p.Pro189Ser),SAMD11,"P10S, P189S",not provided,VCV001648427,1,861349,1,925969,ClinVar:1648427,...,Likely benign,"Dec 9, 2023","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NM_001385641.1(SAMD11):c.576C>T (p.Cys192=),SAMD11,NaN,not provided,VCV001936488,1,861360,1,925980,ClinVar:1936488,...,Likely benign,"Mar 29, 2021","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,NM_001385641.1(SAMD11):c.582C>T (p.Cys194=),SAMD11,NaN,not provided,VCV001568423,1,861366,1,925986,ClinVar:1568423,...,Likely benign,"Jul 5, 2022","criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451320,"MTHFR, 1081C-T",MTHFR,NaN,Homocystinuria due to methylene tetrahydrofola...,VCV000003526,NaN,NaN,NaN,NaN,ClinVar:3526,...,Pathogenic,"Apr 24, 2000",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451331,"NPC1, IVS16, G-A, -82",NPC1,NaN,"Niemann-Pick disease, type C1",VCV000002975,NaN,NaN,NaN,NaN,ClinVar:2975,...,Pathogenic,"Jul 1, 2001",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451340,"NIPBL, IVS44DS, A-G, +4",NIPBL,NaN,Cornelia de Lange syndrome 1,VCV000002149,NaN,NaN,NaN,NaN,ClinVar:2149,...,Pathogenic,"Oct 1, 2004",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1451349,A114N,NIPAL4,NaN,Autosomal recessive congenital ichthyosis 6,VCV000001730,NaN,NaN,NaN,NaN,ClinVar:1730,...,Pathogenic,"Oct 15, 2004",no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def get_alt_ref(name):
    """
    Extracts the reference (ref) and alternative (alt) allele values from a given string using regular expressions.

    The function works with two main types of patterns:
    1. Genomic variants in the form of "position ref > alt" (e.g., "123 A > T").
    2. Amino acid changes in the form of "A123T" (where A is the original amino acid and T is the altered amino acid).

    It returns the reference and alternative alleles or amino acids depending on the input.

    Parameters:
    -----------
    name : str
        The input string containing genomic or amino acid variant information.

    Returns:
    --------
    tuple
        A tuple containing the reference (ref) and alternative (alt) alleles or amino acids.
        If no match is found, both `ref` and `alt` will be empty strings ("").
    """

    # Regular expression pattern to match two types of variants:
    # 1. Genomic variant: "position ref > alt" (e.g., "123 A > T").
    # 2. Amino acid change: "A123T" (e.g., "A" for Alanine, "T" for Threonine).
    pattern = r"\s*([\d\s]+)([ACGT]+)[>-]([ACGT]+)|^([A-Z])(\d+)([A-Z])$"
    
    # Search for matches in the input string using the defined pattern
    match = re.findall(pattern, name)
    
    # Initialize empty strings for ref and alt
    ref, alt = "", ""

    if match:
        # Clean up any extra spaces around the matched groups
        match = list(map(str.strip, match[0]))
        
        # Separate the matching groups into two categories:
        # 1. Genomic variant groups (position, ref, alt)
        # 2. Amino acid change groups (original amino acid, position, altered amino acid)
        pos_ref_alt = match[0:3]  # Groups for genomic variant (position, ref, alt)
        aa_change = match[3:6]    # Groups for amino acid change (original AA, position, new AA)
        
        # Check if the input corresponds to a genomic variant (e.g., "123 A > T")
        if any(pos_ref_alt):
            ref = pos_ref_alt[1]  # Reference allele (e.g., A)
            alt = pos_ref_alt[2]  # Alternative allele (e.g., T)

        # Check if the input corresponds to an amino acid change (e.g., "A123T")
        elif any(aa_change):
            # Assuming `rev_table` is a dictionary that maps amino acid codes to full names or similar
            ref = ",".join(rev_table[aa_change[0]])  # Original amino acid
            alt = ",".join(rev_table[aa_change[2]])  # Altered amino acid

    return (ref, alt)


In [12]:
ref_alts = [get_alt_ref(name) for name in snv_clinvar_df['Name'].values]
print(len(ref_alts))


1225608


In [13]:
extracted_df = pd.DataFrame(ref_alts, columns=['ref', 'alt'])
extracted_df.index = snv_clinvar_df.index
extracted_df


,ref,alt
1,A,G
70,C,T
71,C,T
72,C,T
73,C,T
...,...,...
1451320,C,T
1451331,G,A
1451340,A,G
1451349,"GCT,GCC,GCA,GCG","AAT,AAC"


In [14]:
snv_clinvar_df = pd.concat([snv_clinvar_df, extracted_df], axis=1)
snv_clinvar_df


,Name,Gene(s),Protein change,Condition(s),Accession,GRCh37Chromosome,GRCh37Location,GRCh38Chromosome,GRCh38Location,VariationID,...,Germline review status,Somatic clinical impact,Somatic clinical impact date last evaluated,Somatic clinical impact review status,Oncogenicity classification,Oncogenicity date last evaluated,Oncogenicity review status,Unnamed: 24,ref,alt
1,NM_001005484.2(OR4F5):c.107A>G (p.Glu36Gly),OR4F5,E36G,not specified,VCV002205837,1,69134,1,69134,ClinVar:2205837,...,"criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,G
70,NM_001385641.1(SAMD11):c.552C>T (p.Ile184=),SAMD11,NaN,not provided,VCV001543320,1,861336,1,925956,ClinVar:1543320,...,"criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,T
71,NM_001385641.1(SAMD11):c.565C>T (p.Pro189Ser),SAMD11,"P10S, P189S",not provided,VCV001648427,1,861349,1,925969,ClinVar:1648427,...,"criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,T
72,NM_001385641.1(SAMD11):c.576C>T (p.Cys192=),SAMD11,NaN,not provided,VCV001936488,1,861360,1,925980,ClinVar:1936488,...,"criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,T
73,NM_001385641.1(SAMD11):c.582C>T (p.Cys194=),SAMD11,NaN,not provided,VCV001568423,1,861366,1,925986,ClinVar:1568423,...,"criteria provided, single submitter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451320,"MTHFR, 1081C-T",MTHFR,NaN,Homocystinuria due to methylene tetrahydrofola...,VCV000003526,NaN,NaN,NaN,NaN,ClinVar:3526,...,no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,T
1451331,"NPC1, IVS16, G-A, -82",NPC1,NaN,"Niemann-Pick disease, type C1",VCV000002975,NaN,NaN,NaN,NaN,ClinVar:2975,...,no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,A
1451340,"NIPBL, IVS44DS, A-G, +4",NIPBL,NaN,Cornelia de Lange syndrome 1,VCV000002149,NaN,NaN,NaN,NaN,ClinVar:2149,...,no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,G
1451349,A114N,NIPAL4,NaN,Autosomal recessive congenital ichthyosis 6,VCV000001730,NaN,NaN,NaN,NaN,ClinVar:1730,...,no assertion criteria provided,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"GCT,GCC,GCA,GCG","AAT,AAC"


In [15]:
snv_clinvar_df.columns


Index(['Name', 'Gene(s)', 'Protein change', 'Condition(s)', 'Accession',
       'GRCh37Chromosome', 'GRCh37Location', 'GRCh38Chromosome',
       'GRCh38Location', 'VariationID', 'AlleleID(s)', 'dbSNP ID',
       'Canonical SPDI', 'Variant type', 'Molecular consequence',
       'Germline classification', 'Germline date last evaluated',
       'Germline review status', 'Somatic clinical impact',
       'Somatic clinical impact date last evaluated',
       'Somatic clinical impact review status', 'Oncogenicity classification',
       'Oncogenicity date last evaluated', 'Oncogenicity review status',
       'Unnamed: 24', 'ref', 'alt'],
      dtype='object')

In [16]:
expected_col_names = ['gene', 'chr', 'pos', 'ref', 'alt', 'variation_id', 'variant_type', 'clinical_significance']

clinvar_cols = ['Gene(s)','GRCh38Chromosome','GRCh38Location','ref','alt','VariationID','Variant type','Germline classification']
snv_clinvar_df = snv_clinvar_df[clinvar_cols]
snv_clinvar_df.columns = expected_col_names
snv_clinvar_df.loc[:, 'variant_type'] = snv_clinvar_df['variant_type'].replace({'single nucleotide variant' : 'snv'})
snv_clinvar_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
1,OR4F5,1,69134,A,G,ClinVar:2205837,snv,Likely benign
70,SAMD11,1,925956,C,T,ClinVar:1543320,snv,Likely benign
71,SAMD11,1,925969,C,T,ClinVar:1648427,snv,Likely benign
72,SAMD11,1,925980,C,T,ClinVar:1936488,snv,Likely benign
73,SAMD11,1,925986,C,T,ClinVar:1568423,snv,Likely benign
...,...,...,...,...,...,...,...,...
1451320,MTHFR,NaN,NaN,C,T,ClinVar:3526,snv,Pathogenic
1451331,NPC1,NaN,NaN,G,A,ClinVar:2975,snv,Pathogenic
1451340,NIPBL,NaN,NaN,A,G,ClinVar:2149,snv,Pathogenic
1451349,NIPAL4,NaN,NaN,"GCT,GCC,GCA,GCG","AAT,AAC",ClinVar:1730,snv,Pathogenic


In [17]:
snp_df = pd.read_csv("snp_result.txt", sep="\t")
snp_df


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
0,8,19953280,A>C,snv,247,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.020612:103:1000Genomes|C:0.:0:ALSPAC|C:0.0...
1,8,19953315,G>A,snv,248,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,LPL,A:0.038257:192:1000Genomes|A:0.06876:265:ALSPA...
2,8,19953495,T>C,snv,249,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.100718:504:1000Genomes|C:0.079917:308:ALSP...
3,8,19953512,->TG,delins,250,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,TG:0.241255:1208:1000Genomes|TG:0.123768:477:A...
4,8,19953649,T>C,snv,251,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.057308:287:1000Genomes|C:0.000778:3:ALSPAC...
...,...,...,...,...,...,...,...,...,...,...
990134,X,46881463,"->GTATTA,GTATTATTA",delins,1556328727,likely-benign,by-frequency;by-alfa;by-cluster,3_prime_UTR_variant,RP2,TAGTAT:0.00344:13:1000Genomes|TAGTATTATTA:0.:0...
990135,X,152701444,"C>A,T",snv,1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...
990136,X,152701444,"C>A,T",snv,1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...
990137,X,152701711,C>T,snv,1556827155,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,T:0.005411:20:1000Genomes|A:0.001736:151:ExAC|...


In [18]:
snp_df['variant_type'].value_counts()

variant_type
snv             802116
delins          141752
del              20611
variant_type     19414
ins               4549
mnv               1697
Name: count, dtype: int64

In [19]:
snp_df[snp_df['variant_type'] == 'variant_type']

,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
50,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
101,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
152,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
203,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
254,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
...,...,...,...,...,...,...,...,...,...,...
989909,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
989960,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
990011,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
990062,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency


In [20]:
snp_df = snp_df[snp_df['variant_type'] != 'variant_type']
snp_df


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
0,8,19953280,A>C,snv,247,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.020612:103:1000Genomes|C:0.:0:ALSPAC|C:0.0...
1,8,19953315,G>A,snv,248,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,LPL,A:0.038257:192:1000Genomes|A:0.06876:265:ALSPA...
2,8,19953495,T>C,snv,249,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.100718:504:1000Genomes|C:0.079917:308:ALSP...
3,8,19953512,->TG,delins,250,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,TG:0.241255:1208:1000Genomes|TG:0.123768:477:A...
4,8,19953649,T>C,snv,251,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.057308:287:1000Genomes|C:0.000778:3:ALSPAC...
...,...,...,...,...,...,...,...,...,...,...
990134,X,46881463,"->GTATTA,GTATTATTA",delins,1556328727,likely-benign,by-frequency;by-alfa;by-cluster,3_prime_UTR_variant,RP2,TAGTAT:0.00344:13:1000Genomes|TAGTATTATTA:0.:0...
990135,X,152701444,"C>A,T",snv,1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...
990136,X,152701444,"C>A,T",snv,1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...
990137,X,152701711,C>T,snv,1556827155,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,T:0.005411:20:1000Genomes|A:0.001736:151:ExAC|...


In [21]:
snp_df.tail(20)


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency
990119,20,2652733,"GGCCTGGGCCTG>-,GGCCTG,GGCCTGGGCCTGGGCCTG,GGCCT...",delins,1555779353,pathogenic,by-frequency;by-alfa;by-cluster,intron_variant;upstream_transcript_variant;2KB...,NOP56;SNORD110;MIR1292;LOC105372505,GGGCCTGGGCCTGGGCCTG:0.:0:ALFA
990120,20,2652733,"GGCCTGGGCCTG>-,GGCCTG,GGCCTGGGCCTGGGCCTG,GGCCT...",delins,1555779353,pathogenic,by-frequency;by-alfa;by-cluster,intron_variant;upstream_transcript_variant;2KB...,NOP56;SNORD110;MIR1292;LOC105372505,GGGCCTGGGCCTGGGCCTG:0.:0:ALFA
990121,19,45409896,"->AT,ATCT,ATTAT,ATTATTAT,ATTATTATTAT",delins,1555785259,benign,by-frequency;by-alfa;by-cluster,genic_downstream_transcript_variant;intron_var...,ERCC1;POLR1G,TATTA:0.03094:43:Korea1K|TATC:0.00575:3:Northe...
990122,20,3916912,"->CCCCCT,CCCCT",delins,1555789541,likely-benign;benign-likely-benign,by-frequency;by-alfa;by-cluster,genic_downstream_transcript_variant;intron_var...,PANK2;MIR103A2,TCCCCC:0.018613:233:GoESP|TCCCC:0.000546:1:Kor...
990123,20,10644276,"->TCAC,TCACAC,TCACACAAACAC,TCACACAC,TCACACACAC...",delins,1555828066,benign,by-frequency;by-alfa;by-cluster,intron_variant,JAG1,CTCACA:0.3396:1309:ALSPAC|CTCACA:0.3433:1273:T...
990124,X,2856800,"TATC>-,TATCTATC,TATCTATCTATC",delins,1555898880,benign,by-frequency;by-alfa;by-cluster,intron_variant,GYG2,ATCTATCT:0.00002:2:GnomAD|-:0.36977:4739:TOMMO...
990125,22,40345589,"GGG>-,G,GG,GGGG,GGGGG,GGGGGG,GGGGGGG",delins,1555903152,benign,by-frequency;by-alfa;by-cluster,upstream_transcript_variant;2KB_upstream_variant,ADSL,-:0.001645:3:Korea1K|-:0.000015:4:TOPMED|GGGGG...
990126,22,40345589,"GGG>-,G,GG,GGGG,GGGGG,GGGGGG,GGGGGGG",delins,1555903152,benign,by-frequency;by-alfa;by-cluster,upstream_transcript_variant;2KB_upstream_variant,ADSL,-:0.001645:3:Korea1K|-:0.000015:4:TOPMED|GGGGG...
990127,22,40345589,"GGG>-,G,GG,GGGG,GGGGG,GGGGGG,GGGGGGG",delins,1555903152,benign,by-frequency;by-alfa;by-cluster,upstream_transcript_variant;2KB_upstream_variant,ADSL,-:0.001645:3:Korea1K|-:0.000015:4:TOPMED|GGGGG...
990128,22,33432596,"GCAT>-,GCATGCAT,GCATGCATGCAT,GCATGCATGCATGCAT",delins,1555916106,likely-benign,by-frequency;by-alfa;by-cluster,genic_upstream_transcript_variant;intron_variant,LARGE1,CATG:0.0052:87:TOMMO|CATGCATGCATGCATGCATGCATGC...


In [22]:
snp_ref_alts = [variant.strip().split(">") for variant in snp_df['variation']]
len(snp_ref_alts)


970725

In [23]:
extracted_snp = pd.DataFrame(snp_ref_alts, columns = ['ref', 'alt'])
extracted_snp.index = snp_df.index
extracted_snp


,ref,alt
0,A,C
1,G,A
2,T,C
3,-,TG
4,T,C
...,...,...
990134,-,"GTATTA,GTATTATTA"
990135,C,"A,T"
990136,C,"A,T"
990137,C,T


In [24]:
snp_df.loc[:, 'snp_id'] = snp_df['snp_id'].apply(lambda sid: f"SNP:{sid}")
snp_df = pd.concat([snp_df, extracted_snp], axis=1)
snp_df


,#chr,pos,variation,variant_type,snp_id,clinical_significance,validation_status,function_class,gene,frequency,ref,alt
0,8,19953280,A>C,snv,SNP:247,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.020612:103:1000Genomes|C:0.:0:ALSPAC|C:0.0...,A,C
1,8,19953315,G>A,snv,SNP:248,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,LPL,A:0.038257:192:1000Genomes|A:0.06876:265:ALSPA...,G,A
2,8,19953495,T>C,snv,SNP:249,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.100718:504:1000Genomes|C:0.079917:308:ALSP...,T,C
3,8,19953512,->TG,delins,SNP:250,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,TG:0.241255:1208:1000Genomes|TG:0.123768:477:A...,-,TG
4,8,19953649,T>C,snv,SNP:251,benign,by-frequency;by-alfa;by-cluster,intron_variant,LPL,C:0.057308:287:1000Genomes|C:0.000778:3:ALSPAC...,T,C
...,...,...,...,...,...,...,...,...,...,...,...,...
990134,X,46881463,"->GTATTA,GTATTATTA",delins,SNP:1556328727,likely-benign,by-frequency;by-alfa;by-cluster,3_prime_UTR_variant,RP2,TAGTAT:0.00344:13:1000Genomes|TAGTATTATTA:0.:0...,-,"GTATTA,GTATTATTA"
990135,X,152701444,"C>A,T",snv,SNP:1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...,C,"A,T"
990136,X,152701444,"C>A,T",snv,SNP:1556826649,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,A:0.02702:102:1000Genomes|T:0.026419:2747:Gnom...,C,"A,T"
990137,X,152701711,C>T,snv,SNP:1556827155,benign,by-frequency;by-alfa;by-cluster,coding_sequence_variant;synonymous_variant,MAGEA3,T:0.005411:20:1000Genomes|A:0.001736:151:ExAC|...,C,T


In [25]:
snp_df.columns

Index(['#chr', 'pos', 'variation', 'variant_type', 'snp_id',
       'clinical_significance', 'validation_status', 'function_class', 'gene',
       'frequency', 'ref', 'alt'],
      dtype='object')

In [26]:
snp_columns = ['gene', '#chr', 'pos', 'ref', 'alt', 'snp_id', 'variant_type', 'clinical_significance']
snp_df = snp_df[snp_columns]
snp_df.columns = expected_col_names
snp_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,C,SNP:247,snv,benign
1,LPL,8,19953315,G,A,SNP:248,snv,benign
2,LPL,8,19953495,T,C,SNP:249,snv,benign
3,LPL,8,19953512,-,TG,SNP:250,delins,benign
4,LPL,8,19953649,T,C,SNP:251,snv,benign
...,...,...,...,...,...,...,...,...
990134,RP2,X,46881463,-,"GTATTA,GTATTATTA",SNP:1556328727,delins,likely-benign
990135,MAGEA3,X,152701444,C,"A,T",SNP:1556826649,snv,benign
990136,MAGEA3,X,152701444,C,"A,T",SNP:1556826649,snv,benign
990137,MAGEA3,X,152701711,C,T,SNP:1556827155,snv,benign


In [27]:
final_df = pd.concat([snp_df, snv_clinvar_df], axis=0)
final_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,C,SNP:247,snv,benign
1,LPL,8,19953315,G,A,SNP:248,snv,benign
2,LPL,8,19953495,T,C,SNP:249,snv,benign
3,LPL,8,19953512,-,TG,SNP:250,delins,benign
4,LPL,8,19953649,T,C,SNP:251,snv,benign
...,...,...,...,...,...,...,...,...
1451320,MTHFR,NaN,NaN,C,T,ClinVar:3526,snv,Pathogenic
1451331,NPC1,NaN,NaN,G,A,ClinVar:2975,snv,Pathogenic
1451340,NIPBL,NaN,NaN,A,G,ClinVar:2149,snv,Pathogenic
1451349,NIPAL4,NaN,NaN,"GCT,GCC,GCA,GCG","AAT,AAC",ClinVar:1730,snv,Pathogenic


In [28]:
final_df.isna().sum()


gene                     615
chr                      399
pos                      399
ref                        0
alt                        0
variation_id               0
variant_type               0
clinical_significance      0
dtype: int64

In [29]:
final_df = final_df.dropna()
final_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,C,SNP:247,snv,benign
1,LPL,8,19953315,G,A,SNP:248,snv,benign
2,LPL,8,19953495,T,C,SNP:249,snv,benign
3,LPL,8,19953512,-,TG,SNP:250,delins,benign
4,LPL,8,19953649,T,C,SNP:251,snv,benign
...,...,...,...,...,...,...,...,...
1448523,MT-TP,MT,15990,C,T,ClinVar:9570,snv,Likely pathogenic
1448524,MT-TP,MT,15992,A,T,ClinVar:690268,snv,Likely benign
1448525,MT-TP,MT,16002,T,C,ClinVar:690271,snv,Pathogenic/Likely pathogenic
1448526,MT-TP,MT,16011,A,G,ClinVar:690273,snv,Likely benign


In [30]:
final_df.loc[:, 'clinical_significance'] = final_df['clinical_significance'].apply(lambda sig: "non-harmful" if "benign" in sig else "harmful")
final_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,C,SNP:247,snv,non-harmful
1,LPL,8,19953315,G,A,SNP:248,snv,non-harmful
2,LPL,8,19953495,T,C,SNP:249,snv,non-harmful
3,LPL,8,19953512,-,TG,SNP:250,delins,non-harmful
4,LPL,8,19953649,T,C,SNP:251,snv,non-harmful
...,...,...,...,...,...,...,...,...
1448523,MT-TP,MT,15990,C,T,ClinVar:9570,snv,harmful
1448524,MT-TP,MT,15992,A,T,ClinVar:690268,snv,non-harmful
1448525,MT-TP,MT,16002,T,C,ClinVar:690271,snv,harmful
1448526,MT-TP,MT,16011,A,G,ClinVar:690273,snv,non-harmful


In [31]:
sum([1 for var in final_df['ref'] if len(var.split(",")) > 1])


0

In [32]:
sum([1 for var in final_df['alt'] if len(var.split(",")) > 1])


387444

In [33]:
healthy_df = final_df.copy()
healthy_df = healthy_df[healthy_df['ref'] != "-"]
healthy_df['alt'] = healthy_df['ref']
healthy_df.loc[:, 'clinical_significance'] = ['healthy'] * healthy_df.shape[0]
healthy_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,A,SNP:247,snv,healthy
1,LPL,8,19953315,G,G,SNP:248,snv,healthy
2,LPL,8,19953495,T,T,SNP:249,snv,healthy
4,LPL,8,19953649,T,T,SNP:251,snv,healthy
5,LPL,8,19953906,C,C,SNP:253,snv,healthy
...,...,...,...,...,...,...,...,...
1448523,MT-TP,MT,15990,C,C,ClinVar:9570,snv,healthy
1448524,MT-TP,MT,15992,A,A,ClinVar:690268,snv,healthy
1448525,MT-TP,MT,16002,T,T,ClinVar:690271,snv,healthy
1448526,MT-TP,MT,16011,A,A,ClinVar:690273,snv,healthy


In [34]:
final_df = pd.concat([final_df, healthy_df], axis=0)
final_df = final_df.drop_duplicates().reset_index().drop("index", axis=1)
final_df


,gene,chr,pos,ref,alt,variation_id,variant_type,clinical_significance
0,LPL,8,19953280,A,C,SNP:247,snv,non-harmful
1,LPL,8,19953315,G,A,SNP:248,snv,non-harmful
2,LPL,8,19953495,T,C,SNP:249,snv,non-harmful
3,LPL,8,19953512,-,TG,SNP:250,delins,non-harmful
4,LPL,8,19953649,T,C,SNP:251,snv,non-harmful
...,...,...,...,...,...,...,...,...
4050448,MT-TP,MT,15990,C,C,ClinVar:9570,snv,healthy
4050449,MT-TP,MT,15992,A,A,ClinVar:690268,snv,healthy
4050450,MT-TP,MT,16002,T,T,ClinVar:690271,snv,healthy
4050451,MT-TP,MT,16011,A,A,ClinVar:690273,snv,healthy


In [70]:
final_df.to_csv("Mutation_and_healthy_data.csv", index=False)
